## 准备数据

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0

    return (x, y), (x_test, y_test)

In [2]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [6]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        # 第一层权重和偏置 (784 -> 128)
        self.W1 = tf.Variable(tf.random.normal([784, 128], stddev=0.1), trainable=True)
        self.b1 = tf.Variable(tf.zeros([128]), trainable=True)

        # 第二层权重和偏置 (128 -> 10)
        self.W2 = tf.Variable(tf.random.normal([128, 10], stddev=0.1), trainable=True)
        self.b2 = tf.Variable(tf.zeros([10]), trainable=True)

    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        # 输入 x 形状: (batch_size, 28, 28)

        # 展平输入，从 (batch_size, 28, 28) 到 (batch_size, 784)
        x_flat = tf.reshape(x, [-1, 784])  # -1 表示自动推断 batch_size

        # 第一层：线性变换 + ReLU
        h1 = tf.matmul(x_flat, self.W1) + self.b1  # (batch_size, 128)
        h1 = tf.nn.relu(h1)

        # 第二层：线性变换得到 logits
        logits = tf.matmul(h1, self.W2) + self.b2  # (batch_size, 10)

        return logits

model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [9]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.1*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [10]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer,
                                    tf.constant(train_data[0], dtype=tf.float32),
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model,
                      tf.constant(test_data[0], dtype=tf.float32),
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 1.9475522 ; accuracy 0.3744
epoch 1 : loss 1.8655021 ; accuracy 0.43176666
epoch 2 : loss 1.7890692 ; accuracy 0.48031667
epoch 3 : loss 1.7176011 ; accuracy 0.5222333
epoch 4 : loss 1.6506388 ; accuracy 0.55478334
epoch 5 : loss 1.5878128 ; accuracy 0.58178335
epoch 6 : loss 1.528856 ; accuracy 0.60566664
epoch 7 : loss 1.4735084 ; accuracy 0.62691665
epoch 8 : loss 1.4215324 ; accuracy 0.6447333
epoch 9 : loss 1.3727349 ; accuracy 0.66048336
epoch 10 : loss 1.3269379 ; accuracy 0.67513335
epoch 11 : loss 1.2839625 ; accuracy 0.6878333
epoch 12 : loss 1.2436486 ; accuracy 0.69946665
epoch 13 : loss 1.2058303 ; accuracy 0.70998335
epoch 14 : loss 1.1703559 ; accuracy 0.71888334
epoch 15 : loss 1.1370738 ; accuracy 0.72713333
epoch 16 : loss 1.1058396 ; accuracy 0.73508334
epoch 17 : loss 1.0765121 ; accuracy 0.74368334
epoch 18 : loss 1.0489576 ; accuracy 0.7511333
epoch 19 : loss 1.0230587 ; accuracy 0.75685
epoch 20 : loss 0.9987052 ; accuracy 0.76295
epoch 21 : loss 0